In [1]:
from googlesearch import search
from selectorlib import Extractor
import requests
import random
import pandas as pd

In [2]:
def google_query(isbn):
    """This function receive a book's ISBN and return an amazon link by looking on Google"""
    
    query = "https://www.amazon.it"+"/"+isbn+"/"
    googleResults = []
    for result in search(query,tld="co.in",num=1,stop=1, pause=2):
        googleResults.append(result)

    return googleResults[0]

In [3]:
def get_HTML(url):
    """This function return the HTML of a given link as a string"""
    rand = float(random.randint(1,100)) # We need to randomaize the request
    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/' + str(rand)+ 'Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko)' + 'Chrome/'+ str(rand) + 'Safari/'+str(rand),
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }
    
    return requests.get(url, headers=headers).text

In [4]:
ISBN = '8871527747'
amazon_link = google_query(ISBN)
print(amazon_link)

https://www.amazon.it/povero-Quello-ricchi-insegnano-denaro/dp/8871527747


In [5]:
book_webPage = get_HTML(amazon_link)
#print(book_webPage)

In [6]:
#Create an object extractor based on the CSS "selectors" specified into "data_selectors.yml"
e = Extractor.from_yaml_file('./data_selectors.yml')
data = e.extract(book_webPage)

#Visualize Book's data:
for k,v in data.items():
    if k != 'link_to_all_reviews':
        print(f"\n-{k.upper()}: {v}")


-TITLE: Padre ricco padre povero. Quello che i ricchi insegnano ai figli sul denaro

-PRICE: 11,87 €

-RATING_AVERAGE: 4,6 su 5 stelle

-NUMBER_OF_REVIEWS: 2.368 voti
